# Titanic competition with TensorFlow Decision Forests

This notebook will take you through the steps needed to train a baseline Gradient Boosted Trees Model using TensorFlow Decision Forests and creating a submission on the Titanic competition. 

This notebook shows:

1. How to do some basic pre-processing. For example, the passenger names will be tokenized, and ticket names will be splitted in parts.
1. How to train a Gradient Boosted Trees (GBT) with default parameters
1. How to train a GBT with improved default parameters
1. How to tune the parameters of a GBTs
1. How to train and ensemble many GBTs

# Imports dependencies

In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.2.0


# Load dataset

In [2]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Prepare dataset

We will apply the following transformations on the dataset.

1. Tokenize the names. For example, "Braund, Mr. Owen Harris" will become ["Braund", "Mr.", "Owen", "Harris"].
2. Extract any prefix in the ticket. For example ticket "STON/O2. 3101282" will become "STON/O2." and 3101282.

In [3]:
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    # takes the last item of the ticket array; should always give an integer
    def ticket_number(x):
        return x.split(" ")[-1]
    
    # creates a new string from the remaining ticket array
    # concatenates if necessary
    # make sure to not return empty strings, but rather NONE
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


Let's keep the list of the input features of the model. Notably, we don't want to train our model on the "PassengerId" and "Ticket" features.

In [4]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived") # this is the outcome
input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_item']


# Convert Pandas dataset to TensorFlow Dataset

In [5]:
def tokenize_names(features, labels=None):
    """Divide the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

# Train model with default parameters

### Train model

First, we are training a GradientBoostedTreesModel model with the default parameters.

In [6]:
model = tfdf.keras.GradientBoostedTreesModel(
    # let's increase verbosity to get some insights
    verbose=0, # increased to 5
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[INFO 2024-02-14T14:54:58.722471923+00:00 kernel.cc:1214] Loading model from path /tmp/tmpy9ffdvup/model/ with prefix 4ab2dec8ead04109
[INFO 2024-02-14T14:54:58.732441497+00:00 abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 2024-02-14T14:54:58.732665116+00:00 kernel.cc:1046] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Accuracy: 0.8152173757553101 Loss:0.8657673597335815


# Train model with improved default parameters

Now you'll use some specific parameters when creating the GBT model

In [7]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=1, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner,
    
    min_examples=1,
    categorical_algorithm="RANDOM",
    max_depth=4, #original disabled
    shrinkage=0.2, #from 0.05)
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=1400, # increase to 4000 from 2000 -> worse accuracy
    #validation_ratio=0.0,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Resolve hyper-parameter template "benchmark_rank1@v1" to "benchmark_rank1@v1" -> {'growing_strategy': 'BEST_FIRST_GLOBAL', 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}.
Use /tmp/tmplv10a9oj as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.316976. Found 891 examples.
Training model...
Model trained in 0:00:00.360884
Compiling model...


[INFO 2024-02-14T14:55:03.364733908+00:00 decision_forest.cc:661] Model loaded with 12 root(s), 356 node(s), and 9 input feature(s).
[INFO 2024-02-14T14:55:03.36738903+00:00 decision_forest.cc:661] Model loaded with 12 root(s), 356 node(s), and 9 input feature(s).
[INFO 2024-02-14T14:55:03.367668445+00:00 [INFO decision_forest.cc:2024-02-14T14:55:03.367711472+00:00661] Model loaded with  12 root(s), decision_forest.cc:661] Model loaded with 12 root(s), 356 node(s), and 9 input feature(s).356 node(s), and 9 input feature(s).

[INFO 2024-02-14T14:55:03.368701837+00:00 decision_forest.cc:661] Model loaded with 12 root(s), 356 node(s), and 9 input feature(s).
[INFO 2024-02-14T14:55:03.368911215+00:00 decision_forest.cc:661] Model loaded with 12 root(s), 356 node(s), and 9 input feature(s).
[INFO 2024-02-14T14:55:03.36896939+00:00 decision_forest.cc:661] Model loaded with 12 root(s), 356 node(s), and 9 input feature(s).
[[[[INFOINFO  INFO 2024-02-14T14:55:03.370020821+00:002024-02-14T14:55:

Model compiled.
Accuracy: 0.782608687877655 Loss:0.958136260509491


Let's look at the model and you can also notice the information about variable importance that the model figured out

In [8]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (10):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.         "Sex"  0.470588 ################
    2.         "Age"  0.350255 ########
    3.        "Fare"  0.333764 #######
    4.        "Name"  0.242640 ##
    5. "Ticket_item"  0.214788 
    6.    "Embarked"  0.205660 
    7.       "Parch"  0.204874 

Variable Importance: MEAN_DECREASE_IN_ACCURACY:
    1.         "Sex"  0.152174 ################
    2.        "Fare"  0.119565 #############
    3.        "Name"  0.032609 #####
    4.       "Parch"  0.010870 ###
    5.      

# Make predictions

In [9]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1


# Training a model with hyperparameter tunning

Hyper-parameter tuning is enabled by specifying the tuner constructor argument of the model. The tuner object contains all the configuration of the tuner (search space, optimizer, trial and objective).


In [10]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15, 0.2])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=1)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

Use /tmp/tmpgnreeysa as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.454619. Found 891 examples.
Training model...
Model trained in 0:01:54.560601
Compiling model...


[INFO 2024-02-14T14:57:00.370535027+00:00 kernel.cc:1214] Loading model from path /tmp/tmpgnreeysa/model/ with prefix 88ec59f00d304709
[INFO 2024-02-14T14:57:00.389175195+00:00 decision_forest.cc:661] Model loaded with 16 root(s), 452 node(s), and 12 input feature(s).
[INFO 2024-02-14T14:57:00.389236212+00:00 abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 2024-02-14T14:57:00.389270176+00:00 kernel.cc:1046] Use fast generic engine


Model compiled.
Accuracy: 0.8767123222351074 Loss:0.6485145092010498


In the last line in the cell above, you can see the accuracy is higher than previously with default parameters and parameters set by hand.

This is the main idea behing hyperparameter tuning.

For more information you can follow this tutorial: [Automated hyper-parameter tuning](https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab)

In [11]:
tuned_model

# Making an ensemble

Here you'll create 100 models with different seeds and combine their results

This approach removes a little bit the random aspects related to creating ML models

In the GBT creation is used the `honest` parameter. It will use different training examples to infer the structure and the leaf values. This regularization technique trades examples for bias estimates.

In [12]:
predictions = None
num_predictions = 0
model = tuned_model # let's try this with the tuned model, seems odd that here it starts over without the tuned model?

for i in range(100):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    #model = tfdf.keras.GradientBoostedTreesModel(
    #    verbose=0, # Very few logs
    #    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    #    exclude_non_specified_features=True, # Only use the features in "features"

        #min_examples=1,
        #categorical_algorithm="RANDOM",
        ##max_depth=4,
        #shrinkage=0.05,
        ##num_candidate_attributes_ratio=0.2,
        #split_axis="SPARSE_OBLIQUE",
        #sparse_oblique_normalization="MIN_MAX",
        #sparse_oblique_num_projections_exponent=2.0,
        #num_trees=2000,
        ##validation_ratio=0.0,
    #    random_seed=i,
    #    honest=True,
    #)
    
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)

i:0
Reading training dataset...
Training dataset read in 0:00:00.053579. Found 891 examples.
Training model...
Model trained in 0:01:55.199081
Compiling model...


[INFO 2024-02-14T14:58:56.221642833+00:00 kernel.cc:1214] Loading model from path /tmp/tmpgnreeysa/model/ with prefix 88ec59f00d304709
[INFO 2024-02-14T14:58:56.234293927+00:00 decision_forest.cc:661] Model loaded with 16 root(s), 452 node(s), and 12 input feature(s).
[INFO 2024-02-14T14:58:56.234340362+00:00 abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 2024-02-14T14:58:56.234382391+00:00 kernel.cc:1046] Use fast generic engine


Model compiled.
i:1
Reading training dataset...
Training dataset read in 0:00:00.053041. Found 891 examples.
Training model...
Model trained in 0:01:54.765929
Compiling model...


[INFO 2024-02-14T15:00:51.512074042+00:00 kernel.cc:1214] Loading model from path /tmp/tmpgnreeysa/model/ with prefix 88ec59f00d304709
[INFO 2024-02-14T15:00:51.525891121+00:00 decision_forest.cc:661] Model loaded with 16 root(s), 452 node(s), and 12 input feature(s).
[INFO 2024-02-14T15:00:51.525936824+00:00 abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 2024-02-14T15:00:51.525967105+00:00 kernel.cc:1046] Use fast generic engine


Model compiled.
i:2
Reading training dataset...
Training dataset read in 0:00:00.046016. Found 891 examples.
Training model...
Model trained in 0:01:55.406412
Compiling model...


...

Model compiled.
i:98
Reading training dataset...
Training dataset read in 0:00:00.044291. Found 891 examples.
Training model...
Model trained in 0:01:55.698891
Compiling model...


[INFO 2024-02-14T18:18:47.94526599+00:00 kernel.cc:1214] Loading model from path /tmp/tmpgnreeysa/model/ with prefix 88ec59f00d304709
[INFO 2024-02-14T18:18:47.962042054+00:00 decision_forest.cc:661] Model loaded with 16 root(s), 452 node(s), and 12 input feature(s).
[INFO 2024-02-14T18:18:47.962108648+00:00 abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 2024-02-14T18:18:47.962147369+00:00 kernel.cc:1046] Use fast generic engine


Model compiled.
i:99
Reading training dataset...
Training dataset read in 0:00:00.058430. Found 891 examples.
Training model...
Model trained in 0:01:55.156858
Compiling model...


[INFO 2024-02-14T18:20:43.728499241+00:00 kernel.cc:1214] Loading model from path /tmp/tmpgnreeysa/model/ with prefix 88ec59f00d304709
[INFO 2024-02-14T18:20:43.77355532+00:00 decision_forest.cc:661] Model loaded with 16 root(s), 452 node(s), and 12 input feature(s).
[INFO 2024-02-14T18:20:43.773624167+00:00 abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 2024-02-14T18:20:43.773660662+00:00 kernel.cc:1046] Use fast generic engine


Model compiled.
Submission exported to /kaggle/working/submission.csv


# What is next

If you want to learn more about TensorFlow Decision Forests and its advanced features, you can follow the official documentation [here](https://www.tensorflow.org/decision_forests) 